In [ ]:
import json
import os


import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.express as px
from pandas import DataFrame


from otld import combine_appended_files
from otld.paths import input_dir

In [ ]:
federal, state = combine_appended_files.main()
crosswalk = json.load(open(os.path.join(input_dir, "column_dict_196.json")))

In [ ]:
federal.head()

In [ ]:
state.head()

In [ ]:
def get_data(level: str) -> DataFrame:
    return globals()[level]


def px_line(df: str, state: str | list[str], column: str):
    df = df.lower()
    df = get_data(df).copy()

    if isinstance(state, str):
        df = df[df["STATE"] == state]
    elif isinstance(state, (list, tuple)):
        df = df[df["STATE"].isin(state)]

    df.loc[:, "STATE"] = df.loc[:, "STATE"].map(lambda x: x.title())

    fig = px.line(
        df,
        "year",
        column,
        color="STATE",
        labels={"year": "Year", column: crosswalk[column], "STATE": "State"},
        range_x=[],
    )
    fig.update_layout(xaxis={"dtick": 1})
    # fig.show()

    return fig

In [ ]:
def display_line(func, state, column):
    display(
        widgets.VBox(
            [
                widgets.HBox([state, column]),
                func(df=federal, state=state.value, column=column.value),
            ]
        )
    )
    # display(func(**kwargs))
    # return func(**kwargs)

In [ ]:
state_select = widgets.SelectMultiple(
    options=federal["STATE"].unique(), value=["U.S. TOTAL"]
)
column_dropdown = widgets.Dropdown(options=federal.columns.to_list(), value="5")
df_select = widgets.Dropdown(options=["State", "Federal"], value="State")

widgets_output = widgets.Output()
display(widgets_output)

line = widgets.HBox([df_select, state_select, column_dropdown])
widgets_output.append_display_data(line)

graph_output = widgets.Output()
display(graph_output)


def handle_change(change):
    with graph_output:
        clear_output()
        fig = px_line(df_select.value, state_select.value, column_dropdown.value)
        fig.show()


df_select.observe(handle_change, names="value")
state_select.observe(handle_change, names="value")
column_dropdown.observe(handle_change, names="value")